In [1]:
%load_ext autoreload
%autoreload 2

# Dependencies

In [2]:
import datasets_handler
import evaluation_metrics
import zeroberto
from zeroberto import ZeroBERTo
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
import gc

/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:166: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def splitDocuments(docs: pd.Series()) -> list():
0it [00:00, ?it/s]


In [6]:
which_dataset = 'folhauol' 
max_inferences = 5000

hyp_template = "Esse texto está relacionado a {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
classes_list = list(raw_data[class_col].unique())

zeroshot_method = "dotproduct"  

zeroshot_config = {
    'similarity_model' : 'sentence-transformers/stsb-xlm-r-multilingual',
    'setfit_model' : 'sentence-transformers/stsb-xlm-r-multilingual',
    'dataset':which_dataset,
    'class_col':class_col,
    'data_col':data_col,
    'split':"zeroshot",
    'method':zeroshot_method,
    'max_inferences':max_inferences,
    'classes':classes_list,
    'template': hyp_template,
    'random_state':422,
    'top_n': 8,
    'n_examples': 8,
    "batch_size" : 4,
    "num_pairs" : 20,
    "num_epochs" : 1,
    'st_train_epochs': 10,
    'st_train_batch_size': 20,
    # Unfreeze the head and freeze the body -> head-only training
    'keep_body_frozen_setfit': True,
    # Unfreeze the head and unfreeze the body -> end-to-end training
    # zeroshot_config['keep_body_frozen_setfit'] = False
}

data_to_label = raw_data.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state']).sort_index()

model = ZeroBERTo(classes_list=zeroshot_config['classes'],hypothesis_template=zeroshot_config['template'],
                  labeling_dataset=data_to_label,labeling_method=zeroshot_config['method'],
                  embeddingModel=zeroshot_config['similarity_model'],config=zeroshot_config)
                  
model.runLabeling()
model.evaluateLabeling()

model.getLabelingMetrics()
model.saveLabelingResults()

### load results from labeling step and create training data for contrastive learning
zeroshot_previous_data = datasets_handler.getZeroshotPreviousData(
    model.config['dataset'],model.config['class_col'],top_n=model.config['top_n'],exec_time=model.config['exec_time'])
raw_data_final, model.config['new_class_col'] = datasets_handler.mergeLabelingToDataset(
    raw_data,zeroshot_previous_data,model.config['class_col'])

df_train, df_test = datasets_handler.splitDataset(raw_data_final,model.config) ### df_test will not be used
train_dataset = datasets_handler.buildDatasetDict(df_train)

### build trainer for contrastive learning
model.buildTrainer(train_dataset)

### train
model.contrastive_train()
gc.collect()

### predict labeling dataset with new classifier
model.getPredictions()
# 5000 sentences --- 4min40s ---sts-xlm-rr
# 5000 sentences ---  ---ricardo-filho/bert-portuguese

setfit_all_metrics = evaluation_metrics.get_metrics(model.y_pred, model.trainer.eval_dataset["class_code"])
print(setfit_all_metrics['weighted'][0])

setfit_exec_time  = evaluation_metrics.saveResults(model.config,setfit_all_metrics)
print(model.config)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
